# Helping the Spacing Guild Navigate to Arrakis
### Spencer Ashton, Curtis Evans, Tyler Sanders, Cannon Tuttle

## Abstract
We find the optimal path and control of navigating from one planet to another planet by using the laws of gravity to help with 
the acceleration. This is done by using Pontryagin maximum principle, and we discovered sensitivity in our system when optimizing over $t_f$ and found a solution to numerical instabilty when 
adding massive planets such as the sun. 

## Background
The Dune Guild Naviators are tasked with delivering House Atreides from the planet Caladan to Arrakis. Usually they solve the optimal control in their mind while taking spice, but since spice production
has been disrupted, they've instead decided to turn to ACME students on Earth to solve the optimal control for them. They have asked the ACME students to find the best path that minimizes fuel consumption over start and end time. 

## Problem Statment and Derivations
### State equations
Since planetary orbits are essentially planar, we simplified the problem by only working in two dimensions. The only elements that affect the ship's motion are the forces of gravity between it and the planets in the system, and its own acceleration control. Newton's Law of Universal Gravitation states that the force of gravity applied on body $2$, exerted by body $1$, is given as follows, where $G$ is the universal gravitational constant.

$\mathbf{F}_{21} = -G\frac{m_1m_2}{||\mathbf{x}_2-\mathbf{x}_1||_2^3}(\mathbf{x}_2-\mathbf{x}_1)$.

We used this equation and Newton's second law of motion, $F=ma$, to solve for the acceleration of the ship due to the gravity applied by a given planet. The net acceleration is given by the sum of the accelerations from all planets in the system, combined with the acceleration due to the ship's control. This defines the state equations, where $m_p$ is the mass of the planet, $\mathbf{x}_p$ is the position vector of the planet, $\mathbf{x}_s$ is the position vector of the spaceship, and $\mathbf{u}$ is the acceleration control vector of the spaceship. The set of all $n$ planets in the system is given as $P = \{p_1, p_2,\dots,p_n\}$ where $p_1$ is the planet Caladan, and $p_2$ is the planet Arrakis. The planets are modeled as point masses, with position and velocity functions defined separately according to their known orbits, assuming the force of gravity from the ship on the planet is negligible.

$\ddot{\mathbf{x}} = \mathbf{u} - G\sum_{p\in{P}}^{}\frac{m_p}{||\mathbf{x}_s-\mathbf{x}_p||_2^3}(\mathbf{x}_s-\mathbf{x}_p)$.

Converting $\ddot{\mathbf{x}}$ into a first order system of equations produces the following state equations. 

$\begin{pmatrix}
    \dot{x}_s \\
    \dot{y}_s \\
    \ddot{x}_s\\
    \ddot{y}_s 
\end{pmatrix} = \begin{pmatrix}
    \dot{x}_s \\
    \dot{y}_s \\
    u_x - G\sum_{p\in{P}}^{}\frac{m_p(x_s - x_p)}{((x_s-x_p)^2+(y_s-y_p)^2)^{3/2}}\\
    u_y - G\sum_{p\in{P}}^{}\frac{m_p(y_s - y_p)}{((x_s-x_p)^2+(y_s-y_p)^2)^{3/2}}
\end{pmatrix}$.

The cost functional is defined to minimize the total fuel usage by minimizing the squared magnitude of the control. 

$J[u] = \int_{t_0}^{t_f}||\mathbf{u}||_2^2dt$.

In order to properly take off and land, the ship must match both the position and velocity of the planet. This gives the following boundary conditions

$\mathbf{x}_s(t_0) = \mathbf{x}_{p_1}(t_0), \quad \dot{\mathbf{x}}_s(t_0) = \dot{\mathbf{x}}_{p_1}(t_0),$

and 

$\mathbf{x}_s(t_f) = \mathbf{x}_{p_2}(t_f),\quad\dot{\mathbf{x}}_s(t_f) = \dot{\mathbf{x}}_{p_2}(t_f)$.

We used the cost functional and state equations to solve the problem in the cases of t0 and tf fixed, as well as t0 and tf free:

$\underset{u}{min}$ $J[u]$,

$\underset{u,t_0,t_f}{min}$ $J[u],$

as the spacing guild wants the launch and landing schedules to be flexible.


### Boundary Value Problem Derivation
Pontryagin's maximum principle tells us that:

$H = \mathbf{p}\cdot\mathbf{f(\mathbf{x})} - L$

$\mathbf{p}' = -\frac{DH}{D\mathbf{x}}$ and

$\frac{DH}{D\tilde{\mathbf{u}}} = 0.$

Applying this to our state equation and Lagrangian yields the following Hamiltonian:

$H = p_1\dot{x}_s + p_2\dot{y}_s + p_3\left[u_x-G\sum_{p\in{P}}^{}\frac{m_p(x_s - x_p)}{((x_s-x_p)^2+(y_s-y_p)^2)^{3/2}}\right] + p_4\left[u_y -G\sum_{p\in{P}}^{}\frac{m_p(y_s - y_p)}{((x_s-x_p)^2+(y_s-y_p)^2)^{3/2}}\right] - u_x^2 - u_y^2.$

This give us the co-state evolution equations:

$\dot{p}_1 = p_3G\left(\sum_{p\in{P}}^{}\frac{m_p}{((x_s-x_p)^2+(y_s-y_p)^2)^{3/2}} - \frac{3m_p(x_s - x_p)^2}{((x_s-x_p)^2+(y_s-y_p)^2)^{5/2}}\right)$

$\dot{p}_2 = p_4G\left(\sum_{p\in{P}}^{}\frac{m_p}{((x_s-x_p)^2+(y_s-y_p)^2)^{3/2}} - \frac{3m_p(y_s - y_p)^2}{((x_s-x_p)^2+(y_s-y_p)^2)^{5/2}}\right)$

$\dot{p}_3 = -p_1 $

$\dot{p}_4 = -p_2. $

Now we solve for the control $\tilde{\mathbf{u}}$ that maximizes the Hamiltonian:

$\frac{\partial H}{\partial u_x} = p_3 - 2u_x = 0$

$\frac{\partial H}{\partial u_y} = p_4 - 2u_y = 0$

[comment]:<$\frac{\partial H}{\partial u_x} = p_3 - 2u_x \:\:\text{and}\:\: \frac{\partial H}{\partial u_y} = p_4 - 2u_y$>

Solving for $u_x$ and $u_y$ yields the optimal control in terms of $\mathbf{p}$,

$\tilde{u}_x = \frac{p_3}{2}$

$\tilde{u}_y = \frac{p_4}{2}.$

[comment]:<$\tilde{u}_x = \frac{p_3}{2} \:\:\text{and}\:\: \tilde{u}_y = \frac{p_4}{2}.$>

Plugging $\tilde{u}_x$ and $\tilde{u}_y$ back into the state and costate evolution equations defines the following boundary value problem.

$\dot{x}_s = \dot{x}_s$

$\dot{y}_s = \dot{y}_s$

$\ddot{x}_s = \frac{p_3}{2}-G\sum_{p\in{P}}^{}\frac{m_p(x_s - x_p)}{((x_s-x_p)^2+(y_s-y_p)^2)^{3/2}}$

$\ddot{y}_s = \frac{p_4}{2}-G\sum_{p\in{P}}^{}\frac{m_p(y_s - y_p)}{((x_s-x_p)^2+(y_s-y_p)^2)^{3/2}}$

$\dot{p}_1 = p_3G\left(\sum_{p\in{P}}^{}\frac{m_p}{((x_s-x_p)^2+(y_s-y_p)^2)^{3/2}} - \frac{3m_p(x_s - x_p)^2}{((x_s-x_p)^2+(y_s-y_p)^2)^{5/2}}\right)$

$\dot{p}_2 = p_4G\left(\sum_{p\in{P}}^{}\frac{m_p}{((x_s-x_p)^2+(y_s-y_p)^2)^{3/2}} - \frac{3m_p(y_s - y_p)^2}{((x_s-x_p)^2+(y_s-y_p)^2)^{5/2}}\right)$

$\dot{p}_3 = -p_1$

$\dot{p}_4 = -p_2.$


These have the boundary conditions:

$x_s(t_0) = x_{p_1}(t_0),\quad y_s = y_{p_1}(t_0),\quad \dot{x}_s(t_0) = \dot{x}_{p_1}(t_0),\quad \dot{y}_s(t_0) = \dot{y}_{p_1}(t_0)$

and 

$x_s(t_f) = x_{p_2}(t_f),\quad y_s = y_{p_2}(t_f),\quad \dot{x}_s(t_f) = \dot{x}_{p_2}(t_f),\quad \dot{y}_s(t_f) = \dot{y}_{p_2}(t_f)$

Since the states $\mathbf{x}_i$ all have fixed boundary conditions, the costates $\mathbf{p}_i$ have no boundary conditions.

If we are optimizing over $t_0$ and $t_f$, we get the added boundary conditions

$H(t_0) = 0$ and $H(t_f) = 0$.

## Methods
* We used solve_bvp etc
* We optimized over t0 and tf simultaneously by doing...
* We defined a planet class to encapsulate...
* Talk about us using logspace and built in functions to improve stability
* Talk about us making a general solver that can take in fixed and free t0, tf, and guesses
* etc
We used scipy.integrate.solve_bvp in order to numerically solve the optimal control problem. We defined a number of classes and functions in order to simplify calculation of various scenarios. These are shown and described below.
### Planets
We modeled each planet's orbit as an ellipse centered at the origin, with velocity, major, and minor axes defined by its observed orbit. We used these to calculate the ellipse's circumference and planet's angular velocity which together define its position at a given time. We modeled the planets as point masses, but kept track of the radius in order to ensure that the spaceship would be at the surface, rather than the center, at the initial and final times. We defined a Planet class to that accepts and stores a given planet's mass, radius, velocity, ellipse major axis, and ellipse minor axis, and calculates its velocity and position vectors at a given time.

In [2]:
#Imports
import numpy as np
from scipy.integrate import solve_bvp
from scipy.integrate._bvp import BVPResult
from matplotlib import pyplot as plt

from typing import List, Tuple, Callable, Optional, Union, Literal

from matplotlib import animation

from scipy.special import ellipe
from operator import itemgetter

from abc import ABC

# Globals
YEAR_TO_SEC = 31536000
BoundaryMode = Union[Literal["both fixed"], Literal["both free"]] # Represent which type of boundary conditions to use.

In [3]:
class Planet:
    """Model an orbitting & offset planet."""
    
    
    def __init__(
        self,
        name: str,
        mass: float,
        R: Optional[Tuple[float, float]],
        velocity: float,
        radius: float = 0,
        t0_years: float = 0,
        offset: Optional[Tuple[float, float]] = None
    ):
        """Create a planet.

        Args:
            name (str): The name.
            mass (float): Mass in KG.
            R (Optional[Tuple[float, float]]): Major and minor radii lengths in KM, or None for no orbitting.
            velocity (float): Speed in m/s of orbit.
            radius (float, optional): Planetary radius in meters. Defaults to 0.
            t0_years (float, optional): initial time offset along orbit, IN YEARS. Defaults to 0.
            offset (Optional[Tuple[float, float]], optional): X and Y offset 
                to orbit center, or None for centered at origin. Defaults to None.
        """
            
        #Initialize parameters
        self.mass = mass
        self.t0_years = t0_years
        self.R = R
        if R is not None:
            self.R1 = R[0]
            self.R2 = R[1]
        self.velocity = velocity
        self.name = name
        self.radius = radius
        
        
        if self.R is not None:

            # calculate the circumference of the ellipse using scipy special ellipse integral
            eccentricity_squared = 1-(self.R2**2)/(self.R1**2)
            Circumference = 4*self.R1*ellipe(eccentricity_squared)
            
            #calculate angular velocity given circumference of orbit, velocity:
            # (m/s)/m = /s, /s *2*pi = radians/s = ω, angular frequency/velocity
            # So we need 2π*velocity/circumference
            self.omega = 2*np.pi*velocity/Circumference #omega is in units of radians/second
            
        self.offset = offset
    
    def px(self,t_sec: np.ndarray) -> np.ndarray:
        """Get the planet's x position over time.

        Args:
            t_sec (np.ndarray): time values, in seconds.

        Returns:
            np.ndarray: x position values, in meters.
        """
        t_new = t_sec + self.t0_years*YEAR_TO_SEC 
        return (
            (np.ones_like(t_new)*self.offset[0] if self.offset is not None else np.zeros_like(t_new))
            + (self.R1*np.cos(self.omega*t_new) if self.R is not None else np.zeros_like(t_new))
        )
    
    def py(self,t_sec: np.ndarray) -> np.ndarray:
        """Get the planet's y position over time.

        Args:
            t_sec (np.ndarray): time values, in seconds.

        Returns:
            np.ndarray: y position values in meters.
        """

        t_new = t_sec + self.t0_years*YEAR_TO_SEC
        return (
            (np.ones_like(t_new)*self.offset[1] if self.offset is not None else np.zeros_like(t_new))
            + (self.R2*np.sin(self.omega*t_new) if self.R is not None else np.zeros_like(t_new))
        )
    
    def vx(self, t_sec: np.ndarray) -> np.ndarray:
        t_new = t_sec + self.t0_years*YEAR_TO_SEC
        if self.R is not None:
            return self.omega*-self.R1*np.sin(self.omega*t_new)
        else:
            return np.zeros_like(t_new)
    
    def vy(self, t_sec: np.ndarray) -> np.ndarray:
        """Get the tangential velocity vector of the planet at a given time.

        Args:
            t_sec (np.ndarray): the times.

        Returns:
            np.ndarray: The X and Y tangential speeds in m/s.
        """
        t_new = t_sec + self.t0_years*YEAR_TO_SEC
        if self.R is not None:
            return self.omega*self.R2*np.cos(self.omega*t_new)
        else:
            return np.zeros_like(t_new)
        

### Animation
In order to easily visualize our solutions, we created an animation function that accepts a solution and animates plots of the trajectory of the spaceship across the solar system on one plot, and the acceleration controls and velocity together on a dual y-axis plot.

In [4]:
def animate(
    animate_timespan: Tuple[float, float],
    t0: float,
    tf: float,
    t: np.ndarray,
    soln: BVPResult,
    planets: List[Planet],
    figname: Optional[str], 
    animname: Optional[str],
    n_anim_frames: int = 200,
    anim_len_sec: float = 5,
    guesses: Optional[Tuple[float, float]] = None,
):
    at0, atf = animate_timespan[0] * YEAR_TO_SEC, animate_timespan[1] * YEAR_TO_SEC
    anim_ts = np.linspace(at0, atf, 10000)

    
    M_TO_KM = 1e-3

    # get the spaceship trajectory & control and convert units.
    sx_real = soln.sol(t)[0]
    sy_real = soln.sol(t)[1]
    vx_real = soln.sol(t)[2]
    vy_real = soln.sol(t)[3]
    # speed_real = np.sqrt(vx_real**2+vy_real**2)
    
    ux_real = soln.sol(t)[6]/2
    uy_real = soln.sol(t)[7]/2
    
    # We need to transform these timeseries so that they span the whole
    # animation range, with the following policy:
    # Before t0, copy planet 0's position and speed, with zero control. 
    # in the middle, use an interpolated value of the position, control, speed, etc...
    # After tf, use planet 1's position and speed, with zero control.
    
    # We will also make the assumption that the animate timespan completely overlaps the 
    # real trajectory timeseries.
    def splice_in_ts(src1, realsrc, src2):
        
        # Create a new timeseries
        s = np.zeros_like(anim_ts)
        
        # Fill the beginning with source 1, up to time t0
        s[anim_ts < t0] = src1[anim_ts < t0]
        
        # Fill the middle with the real values of the ship
        middle_mask = (anim_ts >= t0) & (anim_ts <= tf)
        len_spliced_region = s[middle_mask].shape[0]
        interp_indecies = np.floor(np.linspace(0, realsrc.shape[0]-1, len_spliced_region)).astype(int)
        s[middle_mask] = realsrc[interp_indecies]
        
        # Fill the end with source 2
        s[anim_ts > tf] = src2[anim_ts > tf]
        
        return s
    
    # Transform each ship timeseries so they span the whole range of the animation.
    sx = splice_in_ts(planets[0].px(anim_ts), sx_real, planets[1].px(anim_ts)) * M_TO_KM
    sy = splice_in_ts(planets[0].py(anim_ts), sy_real, planets[1].py(anim_ts)) * M_TO_KM
    vx = splice_in_ts(planets[0].vx(anim_ts), vx_real, planets[1].vx(anim_ts))
    vy = splice_in_ts(planets[0].vy(anim_ts), vy_real, planets[1].vy(anim_ts))
    speed = np.sqrt(vx**2+vy**2)
    ux = splice_in_ts(np.zeros_like(anim_ts), ux_real, np.zeros_like(anim_ts))
    uy = splice_in_ts(np.zeros_like(anim_ts), uy_real, np.zeros_like(anim_ts))
    
    # Create a figure for pictures / animation.
    fig, (ax1, ax2) = plt.subplots(2,1, gridspec_kw={"height_ratios": [2, 1]})
    
    # Rescale the displayed sizes of planets based on their masses.
    masses = [planet.mass for planet in planets]
    min_mass = min(masses)
    max_mass = max(masses)
    min_radius = 5
    max_radius = 12
    def mass_to_radius(mass):
        return min_radius + (mass - min_mass) / (max_mass - min_mass+0.1) * (max_radius - min_radius)
    
    # For each 
    planet_x_ts = []
    planet_y_ts = []
    planet_trajectories = []
    planet_points = []
        
    scaled_t = t0 + (tf - t0) * t
    
    # generate trajectories of the planets and their plots
    for planet in planets:
        
        # Generate planet paths
        pxt = planet.px(anim_ts)
        pyt = planet.py(anim_ts)
        pxt *= M_TO_KM
        pyt *= M_TO_KM
        planet_x_ts.append(pxt)
        planet_y_ts.append(pyt)
        
        # Create plots of planets
        color = next(ax1._get_lines.prop_cycler)['color']
        planet_trajectories.append(ax1.plot(pxt, pyt, label=f'{planet.name}', color=color)[0])
        planet_points.append(ax1.plot(pxt[-1], pyt[-1], "o", color=color, markersize=mass_to_radius(planet.mass))[0])
    
    # Rescale the units of time to be in days.
    
    SEC_TO_DAYS = 365 / YEAR_TO_SEC
    
    t_days = anim_ts * SEC_TO_DAYS
    
    # Create graphs for control and speed.
    control_x_graph, = ax2.plot(t_days, ux, label='x accel.')
    control_y_graph, = ax2.plot(t_days, uy, label='y accel.')
    
    
    ship_color = next(ax1._get_lines.prop_cycler)['color']
    
    speed_axis = ax2.twinx()
    speed_axis.set_ylabel("Speed (m/s)")
    speed_plot, = speed_axis.plot(t_days,speed, color=ship_color, label="Speed")
    
    
    # Graph the spaceship
    ss_point, = ax1.plot(sx[-1], sy[-1], "*", color=ship_color, markersize=min_radius)
    ss_trajectory_plot, = ax1.plot(sx, sy, color=ship_color, label='Spaceship')
    
    # Properly label the figures.
    ax1.set(xlabel="x (km)", ylabel="y (km)", title="Optimal Path")
    ax1.legend(loc="center left", bbox_to_anchor=(1, 0.5, 0.3, 0.3), bbox_transform=ax1.transAxes)
    ax2.set(title='Optimal Control', xlabel="Time (Days)", ylabel="Acceleration ($m/s^2$)")

    ax2.legend(loc="upper left", bbox_to_anchor=(1, -0.32, 0.3, 0.3), bbox_transform=ax2.transAxes)
    speed_axis.legend(loc="lower left", bbox_to_anchor=(1, 1, 0.3, 0.3), bbox_transform=speed_axis.transAxes)
    
    # This is for which day it is, only to be used in the animations.
    time_text = ax1.text(0.01, 0.95, "", fontsize=6, transform=ax1.transAxes)
    
    title_text = f"Fixed-time\n$t_{{forced}}=[{t0 * SEC_TO_DAYS:.0f}, {tf * SEC_TO_DAYS:.0f}]$"
    
    # Add overall title.
    if guesses is not None:
        t0g, tfg = guesses
        t0g = t0g * YEAR_TO_SEC * SEC_TO_DAYS
        tfg = tfg * YEAR_TO_SEC * SEC_TO_DAYS
        title_text = f"Free-time\n$t_{{guess}}=[{t0g:.0f}, {tfg:.0f}]$, $t_{{optimal}}=[{t0 * SEC_TO_DAYS:.0f}, {tf * SEC_TO_DAYS:.0f}]$"
    
    title = fig.suptitle(title_text, fontsize=16)
    
    fig.tight_layout()
    fig.show()
    
    # Save Figure if applicable
    if figname:
        fig.savefig(figname, bbox_inches="tight", dpi=600)
        print(f"Figure saved to {figname}")



    # Now for each animation frame, we update all the x and y values to be partially filled.
    def update_anim(anim_frame):
        
        # This one-liner scales the index into time to account
        # for the number of animation frames vs the number of 
        # actual timesteps present in the time data.
        i = min(anim_ts.shape[0]-1, int((anim_frame+1) / n_anim_frames * anim_ts.shape[0]))
        
        # update spaceship graph
        ss_trajectory_plot.set_xdata(sx[:i+1])
        ss_trajectory_plot.set_ydata(sy[:i+1])
        
        # update spaceship dot
        ss_point.set_xdata(sx[i])
        ss_point.set_ydata(sy[i])
        
        # update planets data
        for j, _ in enumerate(planets):
            planet_trajectories[j].set_xdata(planet_x_ts[j][:i+1])
            planet_trajectories[j].set_ydata(planet_y_ts[j][:i+1])
            
            planet_points[j].set_xdata(planet_x_ts[j][i])
            planet_points[j].set_ydata(planet_y_ts[j][i])
        
        # update control expenditure data
        control_x_graph.set_xdata(t_days[:i+1])
        control_x_graph.set_ydata(ux[:i+1])
        
        control_y_graph.set_xdata(t_days[:i+1])
        control_y_graph.set_ydata(uy[:i+1])
        
        speed_plot.set_xdata(t_days[:i+1])
        speed_plot.set_ydata(speed[:i+1])
        
        # Add which day it is onto animation
        time_text.set_text(f"Day {int(t_days[i])}/{int(t_days[-1])}")
        
        title.set_text(title_text)
        
    # Run and Save Animation if applicable
    if animname: 
        print("Saving animation...")   
        anim = animation.FuncAnimation(fig, update_anim, range(n_anim_frames), interval=anim_len_sec * 1000 // n_anim_frames)
        anim.save(animname, dpi=300)
        print(f"Animation saved to {animname}")

    plt.close()


### Boundary Value Problem Classes
We defined a function that can solve the boundary value problem for both fixed time and free time. This function defines the ODEs given above and the listed boundary conditions, with the free time scenario solving for $t_0$ and $t_f$ as parameters of the ODE with the additional boundary conditions on the Hamiltonian. Due to the astronomical (literally) scale of the numbers involved in the calculations, we defined the ODEs by performing the logarithm of the operations, then taking the exponential of the result, in order to improve the numerical stability of the operations.

Importantly, we made the substitution

$t = t_0 + (t_f - t_0)\hat{t}$

where $\hat{t}\in [0,1]$. This allows us to solve for $t_0$ and $t_f$ as free parameters of the system, while also still working for a fixed $t_0$ and $t_f$.

In [5]:


def best_path_t0_tf(
    planets: List[Planet],
    t0: float,
    tf: float,
    boundary: BoundaryMode,
    y_guess: np.ndarray,
    n_initial_mesh_nodes: int = 10000,
    max_nodes: int = 100000,
    G: float = 6.674e-11,
) -> BVPResult:

    t0 = t0 * YEAR_TO_SEC
    tf = tf * YEAR_TO_SEC
    
    # this stores a list of planets with each planet's mass and position with [mass, pos_x, pos_y]
    
    def extract_t0_tf_from_p(p):
        _t0, _tf = None, None
        if boundary == "both free":
            _t0, _tf = p[0], p[1]
        else:
            _t0, _tf = t0, tf
        return _t0, _tf
    
    def ode(t, y, p):
        '''
        sx: spaceship x position
        sy: spaceship y position
        dsx: spaceship x velocity
        dsy: spaceship y velocity
        fuel: fuel at time t
        p1, p2, p3, p4: costate vars
            '''
        _t0, _tf = extract_t0_tf_from_p(p)
        t_hat = _t0 + (_tf-_t0)*t
        
        
        
        sx, sy = y[0], y[1]
        dsx, dsy = y[2], y[3]
        p1, p2, p3, p4 = y[4], y[5], y[6], y[7]

        ddsx = 0
        ddsy = 0
        dp1 = 0
        dp2 = 0
        
        for planet in planets:
            px_t = planet.px(t_hat)
            py_t = planet.py(t_hat)
            dist = np.linalg.norm(np.array([sx-px_t, sy - py_t]))
            
            dist3 = np.exp(3*np.log(dist))
            dist5 = np.exp(5*np.log(dist))
            dx = sx - planet.px(t_hat)
            dy = sy - planet.py(t_hat)
            
            # Do these operations in log space for numerical stability.
            ddsx += dx*np.exp(np.log(planet.mass) - np.log(dist3))
            ddsy += dy*np.exp(np.log(planet.mass) - np.log(dist3))

            dp1 += np.exp(np.log(planet.mass) - np.log(dist3)) - 3*dx**2*np.exp(np.log(planet.mass) - np.log(dist5))
            dp2 += np.exp(np.log(planet.mass) - np.log(dist3)) - 3*dy**2*np.exp(np.log(planet.mass) - np.log(dist5))

        ddsx = -G * ddsx + p3/2
        ddsy = -G * ddsy + p4/2
        
        dp1 = p3*G*dp1
        dp2 = p4*G*dp2
        
        dp3 = -p1 
        dp4 = -p2 

        return (_tf-_t0)*np.array([dsx, dsy, ddsx, ddsy, dp1, dp2, dp3, dp4])

    def bc(ya, yb, p):
        
        _t0, _tf = extract_t0_tf_from_p(p)
        sxa, sya = ya[0], ya[1]
        dsxa, dsya = ya[2], ya[3]
        p1a, p2a, p3a, p4a = ya[4], ya[5], ya[6], ya[7]

        sxb, syb = yb[0], yb[1]
        dsxb, dsyb = yb[2], yb[3]
        p1b, p2b, p3b, p4b = yb[4], yb[5], yb[6], yb[7]

        ddsxa = 0
        ddsya = 0

        ddsxb = 0
        ddsyb = 0

        for planet in planets:
            dista = np.linalg.norm(np.array([sxa - planet.px(_t0), sya - planet.py(_t0)]))
            dist3a = np.exp(3*np.log(dista))
            dxa = sxa - planet.px(_t0)
            dya = sya - planet.py(_t0)

            distb = np.linalg.norm(np.array([sxb - planet.px(_tf), syb - planet.py(_tf)]))
            dist3b = np.exp(3*np.log(distb))
            dxb = sxb - planet.px(_tf)
            dyb = syb - planet.py(_tf)
            
            # Do these operations in log space for numerical stability.
            ddsxa += dxa*np.exp(np.log(planet.mass) - np.log(dist3a))
            ddsya += dya*np.exp(np.log(planet.mass) - np.log(dist3a))

            ddsxb += dxb*np.exp(np.log(planet.mass) - np.log(dist3b))
            ddsyb += dyb*np.exp(np.log(planet.mass) - np.log(dist3b))

        ha = p1a*dsxa + p2a*dsya + p3a*(-G*ddsxa + p3a/2) + p4a*(-G*ddsya+p4a/2) - (p3a/2)**2 - (p4a/2)**2

        hb = p1b*dsxb + p2b*dsyb + p3b*(-G*ddsxb + p3b/2) + p4b*(-G*ddsyb+p4b/2) - (p3b/2)**2 - (p4b/2)**2
 
        target_start_y = planets[0].py(_t0)
        target_start_x = planets[0].px(_t0) + planets[0].radius
        velocity_start_x = planets[0].vx(_t0)
        velocity_start_y = planets[0].vy(_t0)

        target_end_y = planets[1].py(_tf)
        target_end_x = planets[1].px(_tf) + planets[1].radius
        velocity_end_x = planets[1].vx(_tf)
        velocity_end_y = planets[1].vy(_tf)
        
        return np.array([
            # Start at first planet x, y, with velocity matching the initial planet
            ya[0] - target_start_x,
            ya[1] - target_start_y,
            ya[2] - velocity_start_x,
            ya[3] - velocity_start_y,

            # End at second planet's x/y, with velocity matching the final planet
            yb[0] - target_end_x,
            yb[1] - target_end_y,
            yb[2] - velocity_end_x,
            yb[3] - velocity_end_y,
            
        ]
        + ([
            ha, 
            hb 
        ] if boundary == "both free" else []))
    
    t = np.linspace(0, 1, n_initial_mesh_nodes)

    print("Running solve_bvp...")
    
    p_guess = [t0, tf] if boundary == "both free" else None
    
    soln = solve_bvp(
        (ode if boundary == "both free" else lambda t, y: ode(t, y, None)),
        (bc if boundary == "both free" else lambda ya, yb: bc(ya, yb, None)),
        t,
        y_guess,
        p=p_guess,
        max_nodes=max_nodes
    )
    
    if boundary == "both free":
        t0 = soln.p[0]
        tf = soln.p[1]
    
    return t0, tf, t, soln, planets

## Results and Analysis
TODO:
* Add sup_title to animation making it clear What case it's doing. ie "Fixed time, t0 = {}, tf = {}" or "Free time, Guess: t0 = {}, tf = {}, Result: t0 = {}, tf = {}"
* Keep it displaying days, but the math is slightly screwed up right now and is displaying it as like 1e10 days
* Make it plot planetary movements from [0,t0] with no ship, then make it plot [t0,tf]
* Make animation result smaller (want it big enough to be clear, but small enough to display the whole thing on a laptop screen)
* Talk about sensitivity to initial guesses, how we came up with "reasonable" initial guesses
* Highlight all the cool viable trajectories

We solved for the optimal trajectories and controls for various fixed initial and final times, as well as free initial and final times with various initial guesses. We found that the free time case was relatively sensitive to initial guess. Rather than finding a globally optimal launch and landing time, it found a more optimal time in a small window around the initial guess.

Feel free to play around with planetary characteristics, add planets, change initial guesses, etc!

In [6]:
t_guesses_list = [(.45,.5),(0,.5),(.25,2.5)]
t_anim_spans = [(0.2, 0.8), (0, 1), (0, 4)]
inner_planets_no_sun = [
    Planet(name='Caladan',mass=5.9722e24,R=(147.1e9, 152.1e9), velocity = 29.78e3,t0_years=0,radius=6378e3),
    Planet(name='Arrakis',mass=6.39e23,R=(207e9,249e9), velocity = 24e3,t0_years=.5,radius=3390e3),
    Planet(name='Geidi Prime',mass=3.285e23,R=(46e9,69.82e9), velocity = 47.36e3,t0_years=.55,radius=2440e3),
    Planet(name='Kaitain',mass=4.8675e24,R=(107.48e9,108.94e9), velocity = 35.02e3,t0_years=0.21,radius=6051e3),
    #Planet(name='Sun',mass=1.989e30,R=None, velocity = 0,t0_years=0.0,radius=6051e3)
]

for i,(time_tup, anim_span) in enumerate(zip(t_guesses_list, t_anim_spans)):
    #Test it with the planets class:
    t0_guess,tf_guess = time_tup

    steps = 10000
    y_guess = np.ones((8,10000))*1050

    y_guess[2:] = 1


    animate(
        anim_span,
        *best_path_t0_tf(
            
            inner_planets_no_sun,
            t0=t0_guess, 
            tf=tf_guess,
            boundary="both free",
            y_guess = y_guess,
            max_nodes=30000,
            n_initial_mesh_nodes=steps,
            G=6.674e-11,
        ),
        figname= f"run_{i}_free_time.png",
        animname= f'run_{i}_free_time.mp4',
        guesses=time_tup,
    )
    animate(
        anim_span,
        *best_path_t0_tf(
            inner_planets_no_sun,
            t0=t0_guess, 
            tf=tf_guess,
            boundary="both fixed",
            y_guess = y_guess,
            max_nodes=30000,
            n_initial_mesh_nodes=steps,
            G=6.674e-11,
        ),
        figname=f"run_{i}_fixed_time.png",
        animname=f'run_{i}_fixed_time.mp4'
    )


Running solve_bvp...


/var/folders/t7/fm4bzyds14x6dsdw4f5c82dm0000gn/T/ipykernel_52234/2468920264.py:147: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  fig.show()


Figure saved to run_0_free_time.png
Saving animation...


/var/folders/t7/fm4bzyds14x6dsdw4f5c82dm0000gn/T/ipykernel_52234/2468920264.py:169: MatplotlibDeprecationWarning: Setting data with a non sequence type is deprecated since 3.7 and will be remove two minor releases later
  ss_point.set_xdata(sx[i])
/var/folders/t7/fm4bzyds14x6dsdw4f5c82dm0000gn/T/ipykernel_52234/2468920264.py:170: MatplotlibDeprecationWarning: Setting data with a non sequence type is deprecated since 3.7 and will be remove two minor releases later
  ss_point.set_ydata(sy[i])
/var/folders/t7/fm4bzyds14x6dsdw4f5c82dm0000gn/T/ipykernel_52234/2468920264.py:177: MatplotlibDeprecationWarning: Setting data with a non sequence type is deprecated since 3.7 and will be remove two minor releases later
  planet_points[j].set_xdata(planet_x_ts[j][i])
/var/folders/t7/fm4bzyds14x6dsdw4f5c82dm0000gn/T/ipykernel_52234/2468920264.py:178: MatplotlibDeprecationWarning: Setting data with a non sequence type is deprecated since 3.7 and will be remove two minor releases later
  planet_points[

Animation saved to run_0_free_time.mp4
Running solve_bvp...


...

Animation saved to run_2_free_time.mp4
Running solve_bvp...


/var/folders/t7/fm4bzyds14x6dsdw4f5c82dm0000gn/T/ipykernel_52234/2468920264.py:147: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  fig.show()


Figure saved to run_2_fixed_time.png
Saving animation...


/var/folders/t7/fm4bzyds14x6dsdw4f5c82dm0000gn/T/ipykernel_52234/2468920264.py:169: MatplotlibDeprecationWarning: Setting data with a non sequence type is deprecated since 3.7 and will be remove two minor releases later
  ss_point.set_xdata(sx[i])
/var/folders/t7/fm4bzyds14x6dsdw4f5c82dm0000gn/T/ipykernel_52234/2468920264.py:170: MatplotlibDeprecationWarning: Setting data with a non sequence type is deprecated since 3.7 and will be remove two minor releases later
  ss_point.set_ydata(sy[i])
/var/folders/t7/fm4bzyds14x6dsdw4f5c82dm0000gn/T/ipykernel_52234/2468920264.py:177: MatplotlibDeprecationWarning: Setting data with a non sequence type is deprecated since 3.7 and will be remove two minor releases later
  planet_points[j].set_xdata(planet_x_ts[j][i])
/var/folders/t7/fm4bzyds14x6dsdw4f5c82dm0000gn/T/ipykernel_52234/2468920264.py:178: MatplotlibDeprecationWarning: Setting data with a non sequence type is deprecated since 3.7 and will be remove two minor releases later
  planet_points[

Animation saved to run_2_fixed_time.mp4


In [7]:
!ffmpeg -y -i run_0_fixed_time.mp4 -i run_0_free_time.mp4 -filter_complex hstack run_0_both.mp4
!ffmpeg -y -i run_1_fixed_time.mp4 -i run_1_free_time.mp4 -filter_complex hstack run_1_both.mp4
!ffmpeg -y -i run_2_fixed_time.mp4 -i run_2_free_time.mp4 -filter_complex hstack run_2_both.mp4

ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.3 (clang-1403.0.22.14.1)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --di

<video src="run_0_both.mp4" controls width=800>

<video src="run_1_both.mp4" controls width=800>

<video src="run_2_both.mp4" controls width=800>

### Try it yourself!

## Conclusion
We successfully derived state and costate equations that accurately modeled the dynamics of a spaceship traveling through a solar system while trying to minimize fuel consumption with free time boundaries. We used these equations to set up and numerically solve for a boundary value problem, finding several viable trajectories for the spacing guild's mission. We optimized the code for numerical stability, but the solver is still sensitive to initial guesses when solving for free initial or final times, and sometimes . Specifically we recommend {x} trajectory because {y}. 

## References
[1] William M., Samuel L., Jeff S. University Physics Volume 1. 2021 OpenStax. https://openstax.org/details/books/university-physics-volume-1

[2] Herbert, Frank. Dune. Chilton Books, 1965. 